In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('dataset/cleaned/claims.csv')

C:\Users\sligh\AppData\Local\Temp\ipykernel_26636\4261778006.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dataset/cleaned/claims.csv')


In [4]:
df

,Provider,ClaimID,BeneID,PotentialFraud,isAdmitted,ClaimDaysOpen,DaysHospitalized,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,...,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,IsDead,Age
0,PRV55912,CLM46614,BENE11001,1,1,7,7,26000,PHY390922,NaN,...,1,0,1,1,36000,3204,60,70,0,66
1,PRV55907,CLM66048,BENE11001,0,1,3,3,5000,PHY318495,PHY318495,...,1,0,1,1,36000,3204,60,70,0,66
2,PRV56046,CLM68358,BENE11001,0,1,4,4,5000,PHY372395,NaN,...,1,0,1,1,36000,3204,60,70,0,66
3,PRV52405,CLM38412,BENE11011,0,1,9,9,5000,PHY369659,PHY392961,...,0,0,1,1,5000,1068,250,320,0,95
4,PRV56614,CLM63689,BENE11014,0,1,18,18,10000,PHY379376,PHY398258,...,1,0,0,0,21260,2136,120,100,0,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558206,PRV53699,CLM510792,BENE159198,0,0,1,0,800,PHY364188,PHY364188,...,0,0,1,0,0,0,5470,1870,0,57
558207,PRV53702,CLM551294,BENE159198,0,0,1,0,400,PHY423019,PHY332284,...,0,0,1,0,0,0,5470,1870,0,57
558208,PRV53676,CLM596444,BENE159198,0,0,1,0,60,PHY361063,NaN,...,0,0,1,0,0,0,5470,1870,0,57
558209,PRV53689,CLM636992,BENE159198,0,0,1,0,70,PHY403198,NaN,...,0,0,1,0,0,0,5470,1870,0,57


In [7]:
# Assuming your dataframe is named `df_claims`

# Melt the dataset to combine ClmDiagnosisCode_1 to ClmDiagnosisCode_10 into a single column
melted_df = df.melt(
    id_vars=['Provider', 'DeductibleAmtPaid', 'InscClaimAmtReimbursed'], 
    value_vars=[f'ClmDiagnosisCode_{i}' for i in range(1, 11)], 
    var_name='DiagnosisType', 
    value_name='ClmDiagnosisCode'
)

# Drop rows with missing diagnosis codes
melted_df = melted_df.dropna(subset=['ClmDiagnosisCode'])

# Group by Provider and ClmDiagnosisCode, and calculate count and sums
provider_diagnosis_stats = melted_df.groupby(['Provider', 'ClmDiagnosisCode']).agg(
    Count=('ClmDiagnosisCode', 'size'),
    TotalDeductibleAmtPaid=('DeductibleAmtPaid', 'sum'),
    TotalInscClaimAmtReimbursed=('InscClaimAmtReimbursed', 'sum')
).reset_index()

# Display the result
provider_diagnosis_stats

,Provider,ClmDiagnosisCode,Count,TotalDeductibleAmtPaid,TotalInscClaimAmtReimbursed
0,PRV51001,0088,1,1068.0,3000
1,PRV51001,0388,1,1068.0,12000
2,PRV51001,0416,1,0.0,1500
3,PRV51001,07070,1,1068.0,36000
4,PRV51001,2449,1,0.0,100
...,...,...,...,...,...
945854,PRV57763,V7283,1,0.0,470
945855,PRV57763,V7284,1,0.0,80
945856,PRV57763,V7799,1,0.0,30
945857,PRV57763,V823,1,0.0,40


In [11]:
provider_diagnosis_stats.to_csv('dataset/cleaned/provider_diag_edges.csv', index=False)

In [13]:
# Melt the dataset to combine ClmDiagnosisCode_1 to ClmDiagnosisCode_10 into a single column
melted_df = df.melt(
    id_vars=['BeneID', 'DeductibleAmtPaid', 'InscClaimAmtReimbursed'], 
    value_vars=[f'ClmDiagnosisCode_{i}' for i in range(1, 11)], 
    var_name='DiagnosisType', 
    value_name='ClmDiagnosisCode'
)

# Drop rows with missing diagnosis codes
melted_df = melted_df.dropna(subset=['ClmDiagnosisCode'])

# Group by BeneID and ClmDiagnosisCode, and calculate count and sums
beneficiary_diagnosis_stats = melted_df.groupby(['BeneID', 'ClmDiagnosisCode']).agg(
    Count=('ClmDiagnosisCode', 'size'),
    TotalDeductibleAmtPaid=('DeductibleAmtPaid', 'sum'),
    TotalInscClaimAmtReimbursed=('InscClaimAmtReimbursed', 'sum')
).reset_index()

# Display the result
beneficiary_diagnosis_stats

,BeneID,ClmDiagnosisCode,Count,TotalDeductibleAmtPaid,TotalInscClaimAmtReimbursed
0,BENE100000,71656,1,0.0,70
1,BENE100000,79678,1,0.0,50
2,BENE100001,11590,1,0.0,60
3,BENE100001,2724,1,0.0,300
4,BENE100001,2859,1,0.0,100
...,...,...,...,...,...
1597297,BENE99999,71690,1,0.0,700
1597298,BENE99999,7234,1,0.0,50
1597299,BENE99999,V103,1,0.0,80
1597300,BENE99999,V420,1,0.0,80


In [26]:
beneficiary_diagnosis_stats.to_csv('dataset/cleaned/bene_diag_edges.csv', index=False)

In [25]:
# Melt the dataset for diagnosis codes and physicians
physician_df = df.melt(
    id_vars=['DeductibleAmtPaid', 'InscClaimAmtReimbursed', 'ClmDiagnosisCode_1', 
             'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6', 
             'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10'],
    value_vars=['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician'],
    var_name='PhysicianType',  # This column will exist but is irrelevant
    value_name='Physician'
)

# Drop rows with missing physician values
physician_df = physician_df.dropna(subset=['Physician'])

# Melt the dataset again for diagnosis codes
physician_df = physician_df.melt(
    id_vars=['Physician', 'DeductibleAmtPaid', 'InscClaimAmtReimbursed'],
    value_vars=[f'ClmDiagnosisCode_{i}' for i in range(1, 11)],
    var_name='DiagnosisType',  # This column will exist but is irrelevant
    value_name='ClmDiagnosisCode'
)

# Drop rows with missing diagnosis codes
physician_df = physician_df.dropna(subset=['ClmDiagnosisCode'])

# Group by physician and diagnosis code to calculate metrics
physician_diagnosis_stats = physician_df.groupby(['Physician', 'ClmDiagnosisCode']).agg(
    Count=('ClmDiagnosisCode', 'size'),
    TotalDeductibleAmtPaid=('DeductibleAmtPaid', 'sum'),
    TotalInscClaimAmtReimbursed=('InscClaimAmtReimbursed', 'sum')
).reset_index()

# Display the result
physician_diagnosis_stats

,Physician,ClmDiagnosisCode,Count,TotalDeductibleAmtPaid,TotalInscClaimAmtReimbursed
0,PHY311001,2767,1,0.0,100
1,PHY311001,2859,1,0.0,300
2,PHY311001,4019,1,0.0,100
3,PHY311001,42842,1,0.0,100
4,PHY311001,72409,1,0.0,300
...,...,...,...,...,...
1823077,PHY434158,78799,1,0.0,500
1823078,PHY434158,7921,1,0.0,500
1823079,PHY434158,8840,1,0.0,400
1823080,PHY434158,E9203,1,0.0,400


In [27]:
physician_diagnosis_stats.to_csv('dataset/cleaned/phy_diag_edges.csv', index=False)

In [29]:
# Assuming your dataframe is named `df_claims`

# Melt the dataset to combine ClmDiagnosisCode_1 to ClmDiagnosisCode_10 into a single column
melted_df = df.melt(
    id_vars=['Provider', 'DeductibleAmtPaid', 'InscClaimAmtReimbursed'], 
    value_vars=[f'ClmProcedureCode_{i}' for i in range(1, 4)], 
    var_name='ProcedureType', 
    value_name='ClmProcedureCode'
)

# Drop rows with missing diagnosis codes
melted_df = melted_df.dropna(subset=['ClmProcedureCode'])

# Group by Provider and ClmDiagnosisCode, and calculate count and sums
provider_procedure_stats = melted_df.groupby(['Provider', 'ClmProcedureCode']).agg(
    Count=('ClmProcedureCode', 'size'),
    TotalDeductibleAmtPaid=('DeductibleAmtPaid', 'sum'),
    TotalInscClaimAmtReimbursed=('InscClaimAmtReimbursed', 'sum')
).reset_index()

# Display the result
provider_procedure_stats

,Provider,ClmProcedureCode,Count,TotalDeductibleAmtPaid,TotalInscClaimAmtReimbursed
0,PRV51001,2724.0,1,1068.0,42000
1,PRV51001,3521.0,1,1068.0,42000
2,PRV51001,8659.0,1,1068.0,4000
3,PRV51003,66.0,1,1068.0,8000
4,PRV51003,242.0,1,1068.0,13000
...,...,...,...,...,...
23799,PRV57757,3893.0,1,1068.0,38000
23800,PRV57757,4701.0,1,1068.0,9000
23801,PRV57757,7749.0,1,1068.0,14000
23802,PRV57757,8949.0,1,1068.0,3000


In [30]:
provider_procedure_stats.to_csv('dataset/cleaned/provider_proc_edges.csv', index=False)

In [31]:
# Assuming your dataframe is named `df_claims`

# Melt the dataset to combine ClmDiagnosisCode_1 to ClmDiagnosisCode_10 into a single column
melted_df = df.melt(
    id_vars=['BeneID', 'DeductibleAmtPaid', 'InscClaimAmtReimbursed'], 
    value_vars=[f'ClmProcedureCode_{i}' for i in range(1, 4)], 
    var_name='ProcedureType', 
    value_name='ClmProcedureCode'
)

# Drop rows with missing diagnosis codes
melted_df = melted_df.dropna(subset=['ClmProcedureCode'])

# Group by Provider and ClmDiagnosisCode, and calculate count and sums
bene_procedure_stats = melted_df.groupby(['BeneID', 'ClmProcedureCode']).agg(
    Count=('ClmProcedureCode', 'size'),
    TotalDeductibleAmtPaid=('DeductibleAmtPaid', 'sum'),
    TotalInscClaimAmtReimbursed=('InscClaimAmtReimbursed', 'sum')
).reset_index()

# Display the result
bene_procedure_stats

,BeneID,ClmProcedureCode,Count,TotalDeductibleAmtPaid,TotalInscClaimAmtReimbursed
0,BENE100002,2724.0,1,1068.0,12000
1,BENE100002,3808.0,1,1068.0,12000
2,BENE100006,2762.0,1,1068.0,17000
3,BENE100006,3893.0,1,1068.0,17000
4,BENE100010,2911.0,1,1068.0,4000
...,...,...,...,...,...
29607,BENE99959,5459.0,1,1068.0,16000
29608,BENE99974,3893.0,1,1068.0,11000
29609,BENE99974,4414.0,1,1068.0,11000
29610,BENE99976,5551.0,1,1068.0,14000


In [32]:
bene_procedure_stats.to_csv('dataset/cleaned/bene_proc_edges.csv', index=False)

In [36]:
# Melt the dataset for diagnosis codes and physicians
physician_df = df.melt(
    id_vars=['DeductibleAmtPaid', 'InscClaimAmtReimbursed', 'ClmProcedureCode_1', 
             'ClmProcedureCode_2', 'ClmProcedureCode_3'],
    value_vars=['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician'],
    var_name='PhysicianType',  # This column will exist but is irrelevant
    value_name='Physician'
)

# Drop rows with missing physician values
physician_df = physician_df.dropna(subset=['Physician'])

# Melt the dataset again for diagnosis codes
physician_df = physician_df.melt(
    id_vars=['Physician', 'DeductibleAmtPaid', 'InscClaimAmtReimbursed'],
    value_vars=[f'ClmProcedureCode_{i}' for i in range(1, 4)],
    var_name='ProcedureType',  # This column will exist but is irrelevant
    value_name='ClmProcedureCode'
)

# Drop rows with missing diagnosis codes
physician_df = physician_df.dropna(subset=['ClmProcedureCode'])

# Group by physician and diagnosis code to calculate metrics
physician_proc_stats = physician_df.groupby(['Physician', 'ClmProcedureCode']).agg(
    Count=('ClmProcedureCode', 'size'),
    TotalDeductibleAmtPaid=('DeductibleAmtPaid', 'sum'),
    TotalInscClaimAmtReimbursed=('InscClaimAmtReimbursed', 'sum')
).reset_index()

# Display the result
physician_proc_stats

,Physician,ClmProcedureCode,Count,TotalDeductibleAmtPaid,TotalInscClaimAmtReimbursed
0,PHY311019,64.0,1,1068.0,3000
1,PHY311028,8152.0,1,1068.0,11000
2,PHY311035,8954.0,1,1068.0,9000
3,PHY311056,3522.0,1,1068.0,57000
4,PHY311058,3927.0,1,1068.0,13000
...,...,...,...,...,...
49210,PHY434143,5451.0,1,1068.0,8000
49211,PHY434143,8801.0,1,0.0,10000
49212,PHY434154,4513.0,1,1068.0,5000
49213,PHY434155,70.0,1,1068.0,22000


In [37]:
physician_proc_stats.to_csv('dataset/cleaned/phy_proc_edges.csv', index=False)

In [39]:
# Filter rows where 'DiagnosisGroupCode' is not null
provider_diagnosisgroup_stats = df.dropna(subset=['DiagnosisGroupCode'])

# Group by 'Provider' and 'DiagnosisGroupCode' to calculate metrics
provider_diagnosisgroup_stats = provider_diagnosisgroup_stats.groupby(['Provider', 'DiagnosisGroupCode']).agg(
    Count=('DiagnosisGroupCode', 'size'),
    TotalDeductibleAmtPaid=('DeductibleAmtPaid', 'sum'),
    TotalInscClaimAmtReimbursed=('InscClaimAmtReimbursed', 'sum')
).reset_index()

# Display the result
provider_diagnosisgroup_stats

,Provider,DiagnosisGroupCode,Count,TotalDeductibleAmtPaid,TotalInscClaimAmtReimbursed
0,PRV51001,062,1,1068.0,4000
1,PRV51001,245,1,1068.0,42000
2,PRV51001,353,1,1068.0,3000
3,PRV51001,864,1,1068.0,12000
4,PRV51001,882,1,1068.0,36000
...,...,...,...,...,...
37548,PRV57757,534,1,1068.0,14000
37549,PRV57757,655,1,1068.0,13000
37550,PRV57757,689,1,1068.0,4000
37551,PRV57757,854,1,1068.0,38000


In [40]:
provider_diagnosisgroup_stats.to_csv('dataset/cleaned/pro_diaggrp_edges.csv', index=False)

In [42]:
# Filter rows where 'DiagnosisGroupCode' is not null
bene_diagnosisgroup_stats = df.dropna(subset=['DiagnosisGroupCode'])

# Group by 'Provider' and 'DiagnosisGroupCode' to calculate metrics
bene_diagnosisgroup_stats = bene_diagnosisgroup_stats.groupby(['BeneID', 'DiagnosisGroupCode']).agg(
    Count=('DiagnosisGroupCode', 'size'),
    TotalDeductibleAmtPaid=('DeductibleAmtPaid', 'sum'),
    TotalInscClaimAmtReimbursed=('InscClaimAmtReimbursed', 'sum')
).reset_index()

# Display the result
bene_diagnosisgroup_stats

,BeneID,DiagnosisGroupCode,Count,TotalDeductibleAmtPaid,TotalInscClaimAmtReimbursed
0,BENE100002,302,1,1068.0,12000
1,BENE100004,316,1,1068.0,3000
2,BENE100006,674,1,1068.0,17000
3,BENE100007,630,1,1068.0,4000
4,BENE100010,187,1,1068.0,8000
...,...,...,...,...,...
40416,BENE99965,176,1,1068.0,7000
40417,BENE99974,226,1,1068.0,11000
40418,BENE99976,699,1,1068.0,14000
40419,BENE99984,895,1,1068.0,3000


In [43]:
bene_diagnosisgroup_stats.to_csv('dataset/cleaned/bene_diaggrp_edges.csv', index=False)

In [44]:
# Melt the dataset for diagnosis codes and physicians
physician_df = df.melt(
    id_vars=['DeductibleAmtPaid', 'InscClaimAmtReimbursed', 'DiagnosisGroupCode'],
    value_vars=['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician'],
    var_name='PhysicianType',  # This column will exist but is irrelevant
    value_name='Physician'
)

# Drop rows with missing physician values
physician_df = physician_df.dropna(subset=['Physician'])

physician_df = physician_df.dropna(subset=['DiagnosisGroupCode'])

# Group by 'Provider' and 'DiagnosisGroupCode' to calculate metrics
phy_diagnosisgroup_stats = physician_df.groupby(['Physician', 'DiagnosisGroupCode']).agg(
    Count=('DiagnosisGroupCode', 'size'),
    TotalDeductibleAmtPaid=('DeductibleAmtPaid', 'sum'),
    TotalInscClaimAmtReimbursed=('InscClaimAmtReimbursed', 'sum')
).reset_index()

# Display the result
phy_diagnosisgroup_stats

,Physician,DiagnosisGroupCode,Count,TotalDeductibleAmtPaid,TotalInscClaimAmtReimbursed
0,PHY311002,023,1,1068.0,3000
1,PHY311007,947,1,1068.0,6000
2,PHY311019,641,1,1068.0,3000
3,PHY311023,207,1,1068.0,6000
4,PHY311028,543,1,1068.0,11000
...,...,...,...,...,...
59458,PHY434143,373,1,1068.0,8000
59459,PHY434143,393,1,0.0,10000
59460,PHY434143,939,1,1068.0,17000
59461,PHY434154,350,1,1068.0,5000


In [45]:
phy_diagnosisgroup_stats.to_csv('dataset/cleaned/phy_diaggrp_edges.csv', index=False)

In [ ]:
# Filter rows where 'DiagnosisGroupCode' is not null
provider_diagnosisgroup_stats = df.dropna(subset=['DiagnosisGroupCode'])

# Group by 'Provider' and 'DiagnosisGroupCode' to calculate metrics
provider_diagnosisgroup_stats = provider_diagnosisgroup_stats.groupby(['Provider', 'DiagnosisGroupCode']).agg(
    Count=('DiagnosisGroupCode', 'size'),
    TotalDeductibleAmtPaid=('DeductibleAmtPaid', 'sum'),
    TotalInscClaimAmtReimbursed=('InscClaimAmtReimbursed', 'sum')
).reset_index()

# Display the result
provider_diagnosisgroup_stats